In [1]:
import math
import os
import sys
import pandas as pd
sys.path.append("/home/weber/PycharmProjects/EXOTIC/src")
from statannot import add_stat_annotation

pd.options.mode.chained_assignment = None  # default='warn'
import multiprocessing
import parmap
import numpy as np
import collections
from tqdm import tqdm
import pandarallel
from pandarallel import pandarallel

pandarallel.initialize(nb_workers=20, progress_bar=True)
# tqdm.pandas()
from pprint import pprint
from scipy.stats import zscore
from scipy import stats

import requests
import re
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.colors as mc
from matplotlib.colors import LogNorm
import json
import matplotlib.ticker as mtick
from utils.utils import load_config_file

## YAML FILES CONFIG
yaml = load_config_file(config_file="/home/weber/PycharmProjects/EXOTIC/src/config.yaml")
exotic_files = yaml

import matplotlib.font_manager as font_manager

font_dirs = ['/home/weber/Fonts', ]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
font_list = font_manager.createFontList(font_files)
font_manager.fontManager.ttflist.extend(font_list)

from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']
rcParams['font.weight'] = 'light'

INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


/home/weber/.conda/envs/ExoCarto/lib/python3.7/site-packages/ipykernel_launcher.py:42: MatplotlibDeprecationWarning: 
The createFontList function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use FontManager.addfont instead.


In [99]:
refseq_gtex = pd.read_csv('/gstock/EXOTIC/data/GENOMICS/RefSeq_corrected_by_GTEx.csv.gz', compression='gzip', sep='\t')
refseq_gtex['new_mRNA_exons'] = refseq_gtex['new_mRNA_exons'].apply(eval)
refseq_gtex_mrna_gene = refseq_gtex[['Gene', 'new_mRNA_exons']].explode('new_mRNA_exons').drop_duplicates().groupby('Gene')['new_mRNA_exons'].apply(list).reset_index().rename({ 'new_mRNA_exons' : 'new_mRNA_gene'}, axis=1)
refseq_gtex_mrna_gene['new_mRNA_nb_total'] = refseq_gtex_mrna_gene.new_mRNA_gene.str.len()
refseq_gtex = pd.merge(refseq_gtex, refseq_gtex_mrna_gene, on='Gene')
refseq_gtex['Ratio'] = refseq_gtex['new_mRNA_nb'].astype(str) + '/' + refseq_gtex['new_mRNA_nb_total'].astype(str)

refseq_gtex = refseq_gtex.rename({'ranges' : 'CDS', }, axis=1)
refseq_gtex

,Gene,CDS,new_mRNA_exons,new_mRNA_nb,new_mRNA_gene,new_mRNA_nb_total,Ratio
0,A2M,9220419-9220435,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4
1,A2M,9220779-9220820,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4
2,A2M,9221336-9221438,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4
3,A2M,9222341-9222409,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4
4,A2M,9223084-9223174,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4
...,...,...,...,...,...,...,...
176794,ZZZ3,78047461-78047576,"[NM_001308237, NM_015534]",2,"[NM_001308237, NM_015534]",2,2/2
176795,ZZZ3,78047664-78047811,"[NM_001308237, NM_015534]",2,"[NM_001308237, NM_015534]",2,2/2
176796,ZZZ3,78050202-78050340,"[NM_001308237, NM_015534]",2,"[NM_001308237, NM_015534]",2,2/2
176797,ZZZ3,78097535-78099039,[NM_015534],1,"[NM_001308237, NM_015534]",2,1/2


In [27]:
refseq_corrected = pd.read_parquet(yaml["RefSeq"]["refseq_mrna_cds"]).sort_values(by=['Gene', 'CDS'])
refseq_corrected

,Gene,CDS,mRNA_exons,mRNA_nb,mRNA_gene,mRNA_nb_total,Ratio
0,A1BG,58858388-58858395,[NM_130786],1,[NM_130786],1,1/1
1,A1BG,58858719-58859006,[NM_130786],1,[NM_130786],1,1/1
2,A1BG,58861736-58862017,[NM_130786],1,[NM_130786],1,1/1
3,A1BG,58862757-58863053,[NM_130786],1,[NM_130786],1,1/1
4,A1BG,58863649-58863921,[NM_130786],1,[NM_130786],1,1/1
...,...,...,...,...,...,...,...
212477,ZZZ3,78047461-78047576,"[NM_001308237, NM_015534]",2,"[NM_015534, NM_001308237]",2,2/2
212478,ZZZ3,78047664-78047811,"[NM_001308237, NM_015534]",2,"[NM_015534, NM_001308237]",2,2/2
212479,ZZZ3,78050202-78050340,"[NM_001308237, NM_015534]",2,"[NM_015534, NM_001308237]",2,2/2
212480,ZZZ3,78097535-78099039,[NM_015534],1,"[NM_015534, NM_001308237]",2,1/2


In [106]:
complete_list = list()
for gene in tqdm(refseq_gtex.Gene.unique().tolist()):
    l = list()
    l_start = list()
    l_stop = list()
    refseq_gene = refseq_gtex.loc[refseq_gtex['Gene'] == gene]
    refseq_gene[['CDS_start', 'CDS_stop']] = refseq_gene.CDS.str.split('-', expand=True)
    refseq_gene['Share'] = False

    if len(refseq_gene.CDS_start.unique()) != refseq_gene.shape[0]:
        previous_start, previous_stop, previous_mrnas = 0, 0, []
        for j, r in refseq_gene.iterrows():
            start, stop, mrnas = r['CDS_start'], r['CDS_stop'], r['new_mRNA_exons']
            if previous_start == start:
                
                l_start.append(start)
                
                first_exon = "{}-{}".format(start, previous_stop)
                first_mrnas = list(previous_mrnas) + list(mrnas)
                second_exon = "{}-{}".format(previous_stop, stop)
                second_mrnas = mrnas
                l.append({
                    'Gene' : r['Gene'],
                    'CDS' : first_exon,
                    'new_mRNA_exons' : first_mrnas,
                    'new_mRNA_nb': len(first_mrnas),
                    'new_mRNA_gene' : r['new_mRNA_gene'],
                    'new_mRNA_nb_total' : r['new_mRNA_nb_total'],
                    'Ratio' : "{}/{}".format(len(first_mrnas), r['new_mRNA_nb_total']),
                    'CDS_start' : start,
                    'CDS_stop' : previous_stop,
                    'Share' : True,
                })
                l.append({
                    'Gene' : r['Gene'],
                    'CDS' : second_exon,
                    'new_mRNA_exons' : second_mrnas,
                    'new_mRNA_nb': len(second_mrnas),
                    'new_mRNA_gene' : r['new_mRNA_gene'],
                    'new_mRNA_nb_total' : r['new_mRNA_nb_total'],
                    'Ratio' : "{}/{}".format(len(second_mrnas), r['new_mRNA_nb_total']),
                    'CDS_start' : previous_stop,
                    'CDS_stop' : stop,
                    'Share' : True,                    
                })

            if previous_stop == stop:

                l_stop.append(stop)
                
                first_exon = "{}-{}".format(previous_start, start)
                first_mrnas = list(mrnas)
                second_exon = "{}-{}".format(start, stop)
                second_mrnas = list(previous_mrnas) + list(mrnas)
                l.append({
                    'Gene' : r['Gene'],
                    'CDS' : first_exon,
                    'new_mRNA_exons' : first_mrnas,
                    'new_mRNA_nb': len(first_mrnas),
                    'new_mRNA_gene' : r['new_mRNA_gene'],
                    'new_mRNA_nb_total' : r['new_mRNA_nb_total'],
                    'Ratio' : "{}/{}".format(len(first_mrnas), r['new_mRNA_nb_total']),
                    'CDS_start' : previous_start,
                    'CDS_stop' : start,
                    'Share' : True,
                })
                l.append({
                    'Gene' : r['Gene'],
                    'CDS' : second_exon,
                    'new_mRNA_exons' : second_mrnas,
                    'new_mRNA_nb': len(second_mrnas),
                    'new_mRNA_gene' : r['new_mRNA_gene'],
                    'new_mRNA_nb_total' : r['new_mRNA_nb_total'],
                    'Ratio' : "{}/{}".format(len(second_mrnas), r['new_mRNA_nb_total']),
                    'CDS_start' : start,
                    'CDS_stop' : stop,
                    'Share' : True,                    
                })

            previous_start = start
            previous_stop = stop
            previous_mrnas = mrnas
    
    complete_list.append(pd.concat([refseq_gene.loc[(~refseq_gene['CDS_start'].isin(l_start)) & (~refseq_gene['CDS_stop'].isin(l_stop))], pd.DataFrame(l)], axis=0).sort_values(by='CDS'))
refseq_corrected_with_variable = pd.concat(complete_list).reset_index(drop=True)
refseq_corrected_with_variable['Length'] = refseq_corrected_with_variable['CDS_stop'].astype(int) - refseq_corrected_with_variable['CDS_start'].astype(int)
refseq_corrected_with_variable['Ratio_num'] = refseq_corrected_with_variable['Ratio'].apply(eval)
refseq_corrected_with_variable.loc[refseq_corrected_with_variable['Ratio_num'] < 1, 'Const_Alt'] = 'Alt'
refseq_corrected_with_variable.loc[refseq_corrected_with_variable['Ratio_num'] == 1, 'Const_Alt'] = 'Const'
refseq_corrected_with_variable = pd.merge(refseq_corrected_with_variable, refseq_corrected_with_variable.groupby('Gene')['CDS'].count().reset_index().rename({'CDS' : 'CDS_count'}, axis=1), on='Gene')
refseq_corrected_with_variable.to_parquet('/gstock/EXOTIC/data/GENOMICS/RefSeq_corrected_by_GTEx_with_variable.parquet')

refseq_corrected_with_variable                


100%|██████████| 15352/15352 [03:04<00:00, 83.43it/s]


,Gene,CDS,new_mRNA_exons,new_mRNA_nb,new_mRNA_gene,new_mRNA_nb_total,Ratio,CDS_start,CDS_stop,Share
0,A2M,9220419-9220435,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4,9220419,9220435,False
1,A2M,9220779-9220820,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4,9220779,9220820,False
2,A2M,9221336-9221438,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4,9221336,9221438,False
3,A2M,9222341-9222409,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4,9222341,9222409,False
4,A2M,9223084-9223174,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4,9223084,9223174,False
...,...,...,...,...,...,...,...,...,...,...
177226,ZZZ3,78047461-78047576,"[NM_001308237, NM_015534]",2,"[NM_001308237, NM_015534]",2,2/2,78047461,78047576,False
177227,ZZZ3,78047664-78047811,"[NM_001308237, NM_015534]",2,"[NM_001308237, NM_015534]",2,2/2,78047664,78047811,False
177228,ZZZ3,78050202-78050340,"[NM_001308237, NM_015534]",2,"[NM_001308237, NM_015534]",2,2/2,78050202,78050340,False
177229,ZZZ3,78097535-78099039,[NM_015534],1,"[NM_001308237, NM_015534]",2,1/2,78097535,78099039,False


In [123]:
refseq_corrected_with_variable_tmp = pd.read_parquet('/gstock/EXOTIC/data/GENOMICS/RefSeq_corrected_by_GTEx_with_variable.parquet')
refseq_corrected_with_variable_tmp = refseq_corrected_with_variable_tmp.rename({'CDS' : 'ranges'}, axis=1)
refseq_corrected_with_variable_tmp.to_parquet('/gstock/EXOTIC/data/GENOMICS/RefSeq_corrected_by_GTEx_with_variable.parquet')
refseq_corrected_with_variable_tmp

,Gene,ranges,new_mRNA_exons,new_mRNA_nb,new_mRNA_gene,new_mRNA_nb_total,Ratio,CDS_start,CDS_stop,Share,Length,Ratio_num,Const_Alt,CDS_count
0,A2M,9220419-9220435,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4,9220419,9220435,False,16,1.0,Const,38
1,A2M,9220779-9220820,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4,9220779,9220820,False,41,1.0,Const,38
2,A2M,9221336-9221438,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4,9221336,9221438,False,102,1.0,Const,38
3,A2M,9222341-9222409,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4,9222341,9222409,False,68,1.0,Const,38
4,A2M,9223084-9223174,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4,9223084,9223174,False,90,1.0,Const,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177226,ZZZ3,78047461-78047576,"[NM_001308237, NM_015534]",2,"[NM_001308237, NM_015534]",2,2/2,78047461,78047576,False,115,1.0,Const,12
177227,ZZZ3,78047664-78047811,"[NM_001308237, NM_015534]",2,"[NM_001308237, NM_015534]",2,2/2,78047664,78047811,False,147,1.0,Const,12
177228,ZZZ3,78050202-78050340,"[NM_001308237, NM_015534]",2,"[NM_001308237, NM_015534]",2,2/2,78050202,78050340,False,138,1.0,Const,12
177229,ZZZ3,78097535-78099039,[NM_015534],1,"[NM_001308237, NM_015534]",2,1/2,78097535,78099039,False,1504,0.5,Alt,12


In [107]:
refseq_corrected_with_variable['Share'].value_counts()

False    166051
True      11180
Name: Share, dtype: int64

In [69]:
refseq_corrected
refseq_corrected.loc[refseq_corrected['Gene'] == 'AAK1']


,Gene,CDS,mRNA_exons,mRNA_nb,mRNA_gene,mRNA_nb_total,Ratio
181,AAK1,69688799-69688843,[NM_001371577],1,"[NM_014911, NM_001371575, NM_001371577]",3,1/3
182,AAK1,69703001-69703095,"[NM_014911, NM_001371575]",2,"[NM_014911, NM_001371575, NM_001371577]",3,2/3
183,AAK1,69704012-69704116,[NM_001371575],1,"[NM_014911, NM_001371575, NM_001371577]",3,1/3
184,AAK1,69704012-69704122,[NM_014911],1,"[NM_014911, NM_001371575, NM_001371577]",3,1/3
185,AAK1,69706083-69706193,[NM_014911],1,"[NM_014911, NM_001371575, NM_001371577]",3,1/3
186,AAK1,69707992-69708093,"[NM_014911, NM_001371575]",2,"[NM_014911, NM_001371575, NM_001371577]",3,2/3
187,AAK1,69709843-69709944,"[NM_014911, NM_001371575]",2,"[NM_014911, NM_001371575, NM_001371577]",3,2/3
188,AAK1,69723117-69723212,"[NM_014911, NM_001371575]",2,"[NM_014911, NM_001371575, NM_001371577]",3,2/3
189,AAK1,69732701-69732805,"[NM_014911, NM_001371575]",2,"[NM_014911, NM_001371575, NM_001371577]",3,2/3
190,AAK1,69734553-69734710,"[NM_014911, NM_001371575]",2,"[NM_014911, NM_001371575, NM_001371577]",3,2/3


In [73]:
pext_refseq = pd.read_parquet(yaml['EXOTIC']["pext_refseq_path"])
pext_refseq


,HGNC ID,symbol,ensg,Exon,Adipose_Subcutaneous,Adipose_Visceral_Omentum_,AdrenalGland,Artery_Aorta,Artery_Coronary,Artery_Tibial,...,Skin_SunExposed_Lowerleg_,SmallIntestine_TerminalIleum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,WholeBlood,mean_proportion
0,4291.0,GK2,ENSG00000196475,80327693-80329354,NaN,NaN,NaN,NaN,NaN,1.00000,...,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,1.000000
1,1109.0,BRI3,ENSG00000164713,97911108-97911249,0.58244,0.58290,0.62426,0.61856,0.59839,0.59585,...,0.55253,0.60333,0.61408,0.56897,0.489850,0.59181,0.61988,0.61228,0.54971,0.581680
2,1109.0,BRI3,ENSG00000164713,97911663-97911765,0.58244,0.58290,0.62426,0.61856,0.59839,0.59585,...,0.55253,0.60333,0.61408,0.56897,0.489850,0.59181,0.61988,0.61228,0.54971,0.581680
3,1109.0,BRI3,ENSG00000164713,97920423-97920555,0.58244,0.58290,0.62426,0.61856,0.59839,0.59585,...,0.55253,0.60333,0.61408,0.56897,0.489850,0.59181,0.61988,0.61228,0.54971,0.581680
4,1109.0,BRI3,ENSG00000164713,97921951-97922002,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221813,29197.0,CCSER2,ENSG00000107771,86259631-86259715,0.66182,0.63653,0.68397,0.59614,0.64098,0.64226,...,0.53580,0.70740,0.76434,0.66667,0.033532,0.51133,0.62554,0.56584,0.50833,0.668990
221814,29197.0,CCSER2,ENSG00000107771,86259715-86259763,0.66182,0.63653,0.68397,0.59614,0.64098,0.64226,...,0.53580,0.70740,0.76434,0.66667,0.033532,0.51133,0.62554,0.56584,0.50833,0.668990
221815,29197.0,CCSER2,ENSG00000107771,86273205-86273268,0.66182,0.63653,0.68397,0.59614,0.64098,0.64226,...,0.53580,0.70740,0.76434,0.66667,0.034523,0.51133,0.62554,0.56584,0.50833,0.672050
221816,29197.0,CCSER2,ENSG00000107771,86273268-86273299,0.66182,0.63653,0.68397,0.59614,0.64098,0.64226,...,0.53580,0.70740,0.76434,0.66667,0.034523,0.51133,0.62554,0.56584,0.50833,0.672050


In [76]:
pext_refseq.loc[pext_refseq['Exon'].isin(['52573617-52573798', '52573798-52573822'])]

,HGNC ID,symbol,ensg,Exon,Adipose_Subcutaneous,Adipose_Visceral_Omentum_,AdrenalGland,Artery_Aorta,Artery_Coronary,Artery_Tibial,...,Skin_SunExposed_Lowerleg_,SmallIntestine_TerminalIleum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,WholeBlood,mean_proportion
179995,24086.0,A1CF,ENSG00000148584,52573617-52573798,NaN,NaN,1.0,1.0,NaN,1.0,...,1.0,1.0000,NaN,1.0,0.66667,1.0,NaN,NaN,NaN,0.98598
179996,24086.0,A1CF,ENSG00000148584,52573798-52573822,NaN,NaN,0.0,0.0,NaN,0.0,...,0.0,0.1611,NaN,0.0,0.00000,0.0,NaN,NaN,NaN,0.03096


In [111]:
pd.merge(refseq_corrected_with_variable, pext_refseq.rename({'Exon' : 'CDS', 'symbol' : 'Gene'}, axis=1), on=['Gene', 'CDS'])

,Gene,CDS,new_mRNA_exons,new_mRNA_nb,new_mRNA_gene,new_mRNA_nb_total,Ratio,CDS_start,CDS_stop,Share,...,Skin_SunExposed_Lowerleg_,SmallIntestine_TerminalIleum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,WholeBlood,mean_proportion
0,A2M,9220419-9220435,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4,9220419,9220435,False,...,0.66766,0.63152,0.50897,0.60315,0.56368,0.66641,0.67561,0.60225,0.60256,0.63863
1,A2M,9220779-9220820,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4,9220779,9220820,False,...,0.66766,0.63152,0.50897,0.60315,0.56368,0.66641,0.67561,0.60225,0.60256,0.63863
2,A2M,9221336-9221438,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4,9221336,9221438,False,...,0.66766,0.63152,0.50897,0.60315,0.56368,0.66641,0.67561,0.60225,0.60256,0.63863
3,A2M,9222341-9222409,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4,9222341,9222409,False,...,0.66766,0.63152,0.50897,0.60315,0.56368,0.66641,0.67561,0.60225,0.60256,0.63863
4,A2M,9223084-9223174,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,4/4,9223084,9223174,False,...,0.66766,0.63152,0.50897,0.60315,0.56368,0.66641,0.67561,0.60225,0.60256,0.63863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178741,ZZEF1,4015903-4016102,[NM_015113],1,[NM_015113],1,1/1,4015903,4016102,False,...,0.50401,0.59616,0.68027,0.62568,0.41963,0.51624,0.58079,0.46835,0.59546,0.62111
178742,ZZEF1,4017593-4017764,[NM_015113],1,[NM_015113],1,1/1,4017593,4017764,False,...,0.50401,0.59616,0.68027,0.62568,0.41963,0.51624,0.58079,0.46835,0.59546,0.62111
178743,ZZEF1,4020266-4020460,[NM_015113],1,[NM_015113],1,1/1,4020266,4020460,False,...,0.50401,0.59616,0.68027,0.62568,0.41963,0.51624,0.58079,0.46835,0.59546,0.62111
178744,ZZEF1,4027201-4027345,[NM_015113],1,[NM_015113],1,1/1,4027201,4027345,False,...,0.50401,0.59616,0.68027,0.62568,0.41963,0.51624,0.58079,0.46835,0.59546,0.62111
